In [4]:
import pandas as pd
import os

In [6]:
cwd=os.getcwd()

In [7]:
cwd

'/home/shalagodor/code/shalagodor/arb/stock-stat-replica/notebooks'

In [8]:
os.chdir("..")

In [9]:
os.getcwd()

'/home/shalagodor/code/shalagodor/arb/stock-stat-replica'

In [11]:
source_folder='FTSE'

In [12]:
cwd=os.getcwd() + '/data/' + source_folder

In [13]:
cwd

'/home/shalagodor/code/shalagodor/arb/stock-stat-replica/data/FTSE'

In [17]:
filelist=os.listdir(cwd)

In [ ]:
filelist

['KGF.csv',
 'AAL.csv',
 'BLND.csv',
 'HSX.csv',
 'DCC.csv',
 'HWDN.csv',
 'LSEG.csv',
 'BNZL.csv',
 'AHT.csv',
 'MNDI.csv',
 'SVT.csv',
 'ABF.csv',
 'CTEC.csv',
 'NWG.csv',
 'CRDA.csv',
 'DPLM.csv',
 'ANTO.csv',
 'WPP.csv',
 'INF.csv',
 'SDR.csv',
 'EXPN.csv',
 'AZN.csv',
 'SGE.csv',
 'LGEN.csv',
 'CPG.csv',
 'ITRK.csv',
 'HLMA.csv',
 'III.csv',
 'MKS.csv',
 'PHNX.csv',
 'PRU.csv',
 'AUTO.csv',
 'DGE.csv',
 'BEZ.csv',
 'CCH.csv',
 'RKT.csv',
 'RIO.csv',
 'BTRW.csv',
 'ENT.csv',
 'AAF.csv',
 'ADM.csv',
 'LMP.csv',
 'ULVR.csv',
 'PSN.csv',
 'BP.csv',
 'IMB.csv',
 'TSCO.csv',
 'SGRO.csv',
 'HLN.csv',
 'EDV.csv',
 'IHG.csv',
 'FCIT.csv',
 'UU.csv',
 'VOD.csv',
 'SHEL.csv',
 'SN.csv',
 'RR.csv',
 'SMIN.csv',
 'SPX.csv',
 'ICG.csv',
 'RTO.csv',
 'BATS.csv',
 'PSON.csv',
 'WEIR.csv',
 'TW.csv',
 'IAG.csv',
 'STJ.csv',
 'GLEN.csv',
 'LLOY.csv',
 'UTG.csv',
 'BKG.csv',
 'RMV.csv',
 'CNA.csv',
 'HIK.csv',
 'IMI.csv',
 'JD.csv',
 'SMT.csv',
 'PSH.csv',
 'FRES.csv',
 'BARC.csv',
 'AV.csv',
 'STAN

In [37]:
df=pd.read_csv(cwd + '/KGF.csv', index_col=0)

In [38]:
df

,1. open,2. high,3. low,4. close,5. volume
2025-03-07,251.40,259.100,249.60,259.10,4814369
2025-03-06,251.80,254.500,250.20,252.30,3837115
2025-03-05,245.80,253.356,245.80,249.30,2808418
2025-03-04,245.20,246.430,243.00,243.40,3542837
2025-03-03,246.60,249.500,246.10,248.10,5423713
...,...,...,...,...,...
2005-01-10,312.00,314.250,308.25,308.75,8760675
2005-01-07,309.75,313.000,309.75,311.50,11756620
2005-01-06,310.25,311.750,308.75,310.50,18005811
2005-01-05,314.00,314.000,307.50,308.50,19848670


In [41]:
df['1. open']

2025-03-07    251.40
2025-03-06    251.80
2025-03-05    245.80
2025-03-04    245.20
2025-03-03    246.60
               ...  
2005-01-10    312.00
2005-01-07    309.75
2005-01-06    310.25
2005-01-05    314.00
2005-01-04    308.00
Name: 1. open, Length: 5099, dtype: float64

In [61]:
data_read=[]

for f in filelist:
    df=pd.read_csv(cwd + '/'+ f, index_col=0)
    data_read.append(df[['1. open']])

In [62]:
data_read

[            1. open
 2025-03-07   251.40
 2025-03-06   251.80
 2025-03-05   245.80
 2025-03-04   245.20
 2025-03-03   246.60
 ...             ...
 2005-01-10   312.00
 2005-01-07   309.75
 2005-01-06   310.25
 2005-01-05   314.00
 2005-01-04   308.00
 
 [5099 rows x 1 columns],
               1. open
 2025-03-07  2446.0000
 2025-03-06  2418.0000
 2025-03-05  2375.0000
 2025-03-04  2303.0000
 2025-03-03  2335.5000
 ...               ...
 2005-01-10  1139.5761
 2005-01-07  1147.9829
 2005-01-06  1134.9058
 2005-01-05  1124.6309
 2005-01-04  1143.3125
 
 [5099 rows x 1 columns],
              1. open
 2025-03-07  346.4000
 2025-03-06  347.2000
 2025-03-05  346.8000
 2025-03-04  353.8000
 2025-03-03  359.0000
 ...              ...
 2005-01-07  861.4975
 2005-01-06  859.9974
 2005-01-05  874.9975
 2005-01-04  894.9974
 2005-01-03  895.9973
 
 [5107 rows x 1 columns],
             1. open
 2025-03-07  1140.00
 2025-03-06  1186.00
 2025-03-05  1180.00
 2025-03-04  1196.00
 2025-03-03  1178.0

In [64]:
pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (146 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


In [65]:
import requests

In [66]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=BT-A&outputsize=full&apikey=FQ5KRJXRO5J30Y2S'
r = requests.get(url)
data = r.json()

print(data)

{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY.'}


In [92]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=BT-A.L&outputsize=full&apikey=FQ5KRJXRO5J30Y2S'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'BT-A.L', '3. Last Refreshed': '2025-03-07', '4. Output Size': 'Full size', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-03-07': {'1. open': '152.0500', '2. high': '160.8500', '3. low': '151.7000', '4. close': '160.2500', '5. volume': '16834676'}, '2025-03-06': {'1. open': '151.7500', '2. high': '153.0500', '3. low': '150.5505', '4. close': '152.4000', '5. volume': '18469650'}, '2025-03-05': {'1. open': '155.0500', '2. high': '155.6500', '3. low': '151.2500', '4. close': '151.2500', '5. volume': '18772480'}, '2025-03-04': {'1. open': '155.8000', '2. high': '156.2000', '3. low': '154.3000', '4. close': '154.8500', '5. volume': '13650270'}, '2025-03-03': {'1. open': '159.4500', '2. high': '160.1500', '3. low': '156.0000', '4. close': '156.3000', '5. volume': '12280960'}, '2025-02-28': {'1. open': '157.4500', '2. high': '161.2500', '3. low': '157.3500', '4. close': '159.8000'

In [70]:
 data.to_csv(cwd +'/BTA.csv')

AttributeError: 'dict' object has no attribute 'to_csv'

In [21]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=^FTSE&outputsize=full&apikey=8K11APUHBSA4UXJ3'
r = requests.get(url)
data = r.json()


In [22]:
data

{}

In [12]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=^FTSE&apikey=8K11APUHBSA4UXJ3'
r = requests.get(url)
data = r.json()

print(data)

{'bestMatches': []}
